<div>
    <h2><center style="background-color:#00878b; color:white;"> (NOT) AI HYPE CYCLE 2021</center></h2>
</div>   
<p style="text-align:justify;font-size:16px"> Every year the American research, advisory and information technology firm Gartner publishes a report in graphical representation called Gartner Hype Cycle which represents the maturity, adoption, and social application of specific technologies. </p>
    
  <center><img src="https://i.imgur.com/Xnpug2R.png[/img]" width = 700></center> 
  
  <p style="text-align:justify;font-size:16px"> According to Gartner analysis 2021, the following are key takeaway <br>
  <b>Current</b> <br>
 <ol style="font-size:16px">
  <li>Machine Learning </li>
  <li>Deep Learning</li>
  <li>Chatbots</li>
  <li>Natural Language Processing </li>
  <li> AI Cloud Services </li>
  <li> Semantic Search </li>
  <li> Computer Vision </li>
</ol>  
 <b>Near-future focus</b> <br>
 <ol style="font-size:16px">
  <li>Generative AI </li>
  <li>AI Governance, AI Orchestration and Automation Platforms, MLOps, ModelOps</li>
  <li>Responsible AI</li>
</ol>  
 <b>Later in future</b> <br>
 <ol style="font-size:16px">
  <li>Small and Wide Data </li>
  <li>Transformers</li>
  <li>Knowledge Graphs</li>
</ol>  
</p>
<p style="text-align:justify;font-size:16px"> From the Gartner hype cycle for Artificial Intelligence (AI) we can see AI comprises a lot of technologies and the AI world is running at a fast pace. Each year so many new technologies and tools are popping up also, the existing ones are growing rapidly. The objective of this analysis is to understand are we running in equal pace with AI. As mentioned above, machine learning, deep learning, computer vision, natural language processing, cloud services are the current trend and the Gartner team have already predicted what could be the future but, are the commoners or machine learning enthusiasts using these technologies for their work? If not what could be done? Any analysis we would focus on the YES i.e what is being used or prefered but this analysis focuses more on the NO/NONE. Which subset of people doesn't use any particular tool/technology? What could be the reason?  </p>

<p style="text-align:justify;font-size:14px"> The subset among the total respondents for analysis are selected by either their ML experince (Q15 in the survey) or what task does their job involve (Q24 in the survey).</p>

In [ ]:
import numpy as np 
import pandas as pd 
from collections import OrderedDict

import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.offline as py
import plotly.io as pio
import cufflinks as cf
!pip install circlify
import circlify
import matplotlib.pyplot as plt
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
py.init_notebook_mode(connected=True)
%config InlineBackend.figure_format = 'retina' 
pio.templates.default = "plotly_white"
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

import warnings
warnings.filterwarnings("ignore")

data = pd.read_csv('../input/kaggle-data-science-survey-20172021/kaggle_survey_2017_2021.csv',low_memory=False)
data.drop('Time from Start to Finish (seconds)',axis=1,inplace=True)
data.replace('United Kingdom of Great Britain and Northern Ireland','United Kingdom',inplace = True)
data.replace('Viet Nam','Vietnam',inplace=True)
data.replace('Iran, Islamic Republic of...','Iran',inplace=True)
data.rename(columns={'-': 'Year'}, inplace=True)

df_2021 = data[data['Year'] == '2021']
df_2020 = data[data['Year'] == '2020']
df_2019 = data[data['Year'] == '2019']
df_2018 = data[data['Year'] == '2018']
df_2017 = data[data['Year'] == '2017']


df2021 = pd.read_csv('../input/kaggle-survey-2021/kaggle_survey_2021_responses.csv',low_memory=False)
df2021.drop(0,inplace=True)

color = '#00c0c7'
colors = ["#e60049", "#0bb4ff", "#50e991", "#e6d800", "#9b19f5", "#ffa300", "#dc0ab4", "#b3d4ff", "#00bfa0","#0230c7","#00c707","#008b05"]

'''
Adapted from https://www.kaggle.com/paultimothymooney/2020-kaggle-data-science-machine-learning-survey
'''

def count_then_return_percent(dataframe,column_name):
    '''
    A helper function to return value counts as percentages.
    '''
    
    counts = dataframe[column_name].value_counts(dropna=False)
    percentages = round(counts*100/(dataframe[column_name].count()),1)
    return percentages

def count_then_return_percent_for_multiple_column_questions(dataframe,list_of_columns_for_a_single_question,dictionary_of_counts_for_a_single_question):
    '''
    A helper function to convert counts to percentages.
    '''
    
    df = dataframe
    subset = list_of_columns_for_a_single_question
    df = df[subset]
    df = df.dropna(how='all')
    total_count = len(df) 
    dictionary = dictionary_of_counts_for_a_single_question
    for i in dictionary:
        dictionary[i] = round(float(dictionary[i]*100/total_count),1)
    return dictionary

def create_dataframe_of_counts(dataframe,column,rename_index,rename_column,return_percentages=False):
    '''
    A helper function to create a dataframe of either counts 
    or percentages, for a single multiple choice question. 
    '''
    df = dataframe[column].value_counts().reset_index() 
    if return_percentages==True:
        df[column] = (df[column]*100)/(df[column].sum())
    df = pd.DataFrame(df) 
    df = df.rename({'index':rename_index, 'Q3':rename_column}, axis='columns')
    return df

def sort_dictionary_by_percent(dataframe,list_of_columns_for_a_single_question,dictionary_of_counts_for_a_single_question): 
    ''' 
    A helper function that can be used to sort a dictionary.
    
    It is an adaptation of a similar function
    from https://www.kaggle.com/sonmou/what-topics-from-where-to-learn-data-science.
    '''
    dictionary = count_then_return_percent_for_multiple_column_questions(dataframe,
                                                                list_of_columns_for_a_single_question,
                                                                dictionary_of_counts_for_a_single_question)
    dictionary = {v:k    for(k,v) in dictionary.items()}
    list_tuples = sorted(dictionary.items(), reverse=False) 
    dictionary = {v:k for (k,v) in list_tuples}   
    return dictionary


def binarize_data(df):
    '''
    A utility function to binarize the data.
    '''
    df_cols = df.columns
    for col in df.columns:
        if "_Part_" in col or 'OTHER' in col:
            k = df[col].value_counts()
            df[col] = df[col].replace(k.keys()[0],True)
            df[col] = df[col].fillna(False)
            df[col] = df[col].astype('bool')
    return df

def generateColumnNames(qNo,parts):
    '''
    A utility function to generate columns names in the formart 'Q12_Part_1'
    '''
    colNames = []
    for i in range(1,parts+1):
        colNames.append('Q'+str(qNo)+'_Part_'+str(i))
    return colNames

def mergeDict (keys, values):
    '''
    A helper function to merge values of different dictionaries
    '''
    set_keys=[set(list(i)).pop() for i in zip(*keys)]
    set_values=map(list,zip(*values))
    return dict(zip(set_keys,set_values))

def getQ17Count(df):
    q17_count = {
        'Linear or Logistic Regression' : (df['Q17_Part_1'].count()),
        'Decision Trees or Random Forests': (df['Q17_Part_2'].count()),
        'Gradient Boosting Machines' : (df['Q17_Part_3'].count()),
        'Bayesian Approaches' : (df['Q17_Part_4'].count()),
        'Evolutionary Approaches' : (df['Q17_Part_5'].count()),
        'Dense Neural Networks' : (df['Q17_Part_6'].count()),
        'Convolutional Neural Networks' : (df['Q17_Part_7'].count()),
        'Generative Adversarial Networks' : (df['Q17_Part_8'].count()),
        'Recurrent Neural Networks' : (df['Q17_Part_9'].count()),
        'Transformer Networks' : (df['Q17_Part_10'].count()),
        'None' : (df['Q17_Part_11'].count())
    }
    return q17_count


q18_count = {
    'General purpose image/video tools' : (df_2021['Q18_Part_1'].count()),
    'Image segmentation methods': (df_2021['Q18_Part_2'].count()),
    'Object detection methods' : (df_2021['Q18_Part_3'].count()),
    'Image classification and other general purpose networks' : (df_2021['Q18_Part_4'].count()),
    'Generative Networks' : (df_2021['Q18_Part_5'].count()),
    'None' : (df_2021['Q18_Part_6'].count()),
    'Other' : (df_2021['Q18_OTHER'].count())
}

q19_count = {
    'Word embeddings/vectors' : (df_2021['Q19_Part_1'].count()),
    'Encoder-decoder models': (df_2021['Q19_Part_2'].count()),
    'Contextualized embeddings' : (df_2021['Q19_Part_3'].count()),
    'Transformer language models' : (df_2021['Q19_Part_4'].count()),
    'None' : (df_2021['Q19_Part_5'].count()),
    'Other' : (df_2021['Q19_OTHER'].count())
}



In [ ]:
val = (abs(len(df_2021) - len(df_2017)) / len(df_2017)) * 100.0
noOfRespondents = [len(df_2017), len(df_2018), len(df_2019), len(df_2020), len(df_2021)]
fig1 = go.Bar(x=['2017', '2018', '2019', '2020', '2021'], y=noOfRespondents, text=noOfRespondents,
              width=0.4, textposition='auto',
              marker_color=color)

fig2 = go.Indicator(
    domain={'row': 1, 'column': 3},
    value=val,
    mode='number+delta',
    delta={'reference': 0},
    gauge={'axis': {'range': [0, 100]}})

fig = make_subplots(
    rows=1,
    cols=2,
    specs=[[{'type': 'bar'}, {'type': 'indicator'}]],
    subplot_titles=('Kaggle Survery Respondents Trend', '55% Surge from 2017 -> 2021'))
fig.append_trace(fig1, row=1, col=1)
fig.append_trace(fig2, row=1, col=2)
fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()

<p style="text-align:justify;font-size:16px">Kaggle has conducted their survey successfully for 5 years since 2017 which covers a wide range of questions related to data science community. It has seen a <b> increase of 55% </b> of respondents from 2017 to 2021. <i> More the sample, better the insights!</i>   </p>

<div>
    <h2><center style="background-color:#00878b; color:white;">  CONTENTS </center></h2>
</div>

* [Machine Learning Technologies](#1)
* [Specialized Hardware](#2)
* [Cloud Computing](#3)
* [AutoML](#4)
* [Machine Learning Tools](#5)
* [The Quest to Learn](#6)
* [Key Insights](#7)
* [Conclusion](#8)

<div>
    <h2><center style="background-color:#00878b; color:white;">  1. MACHINE LEARNING TECHNOLOGIES </center><a id="1"></a></h2>
</div>

<p style="text-align:justify;font-size:16px">This section focuses on currently trending technologies from the Gartner Hyper Cycle for AI, 2021.   </p>

In [ ]:
df_2021.replace('I do not use machine learning methods','Do not use machine learning methods',inplace = True)
df_exp = df_2021.dropna(subset=['Q15'])
k = df_exp['Q15'].value_counts().sort_values(ascending=False)
k = (k / len(df_exp)) * (100)
k = k.round(0).astype(int)
experience = pd.DataFrame(k.items(), columns=['Experience', '% of respondents'])
trace = go.Funnel(
    y=experience["Experience"],
    x=experience['% of respondents'], marker=dict(color=color))

layout = go.Layout(barmode="group", width=1000, height=500, showlegend=False)
fig = go.Figure(data=[trace], layout=layout)
fig.update_layout(title='Fig 1.1 Machine Learning Methods Usage Experience', width=800,height=500)
fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()


<p style="text-align:justify;font-size:16px"> Figure 1.1 shows that around 59% (39% less than 1 year and 20% with 1-2 years) of the respondents are beginners. Also 16% of them aren't using any ML Methods. This could make the further analysis biased as most of them are beginners. Some of the questions we wish to answer in this section are 
<ol style="font-size:16px">
  <li>Does the respondents prefer deep learning algorithms?</li>
  <li>Are they using Computer Vision and Natural Language Processing Methods?</li>
  <li>Are the beginners interested in/exposed to the new technologies?</li>
</ol></p>


<center><img src="https://i.imgur.com/QqfcZ36.png[/img]" width = 600></center>  
<br>
<p style="text-align:justify;font-size:16px"> With the increase in use cases, interest in the field of AI has gained its attention from the research community. Many existing algorithms have went through and a lot of new ML and DL algorithms have emerged. We say particularly deep learning is gaining much popularity due to it’s supremacy in terms of accuracy when trained with huge amount of data.  The idea of this section is to understand if traditional machine learning algorithms or recent deep learning algorithms are prefered by the respondents</p>



In [ ]:
algos_2021 = sort_dictionary_by_percent(df_2021, generateColumnNames(17, 11), getQ17Count(df_2021))
ml_methods = pd.DataFrame(algos_2021.items(), columns=['Algorithms', '% of respondents'])
trace = go.Bar(y=ml_methods["Algorithms"], x=ml_methods['% of respondents'],
               orientation='h', marker=dict(color=color))
layout = go.Layout(barmode="group", width=1000, height=500, showlegend=False)
fig = go.Figure(data=[trace], layout=layout)
fig.update_layout(title=' Fig 1.2 Machine Learning Algorithms Used',
    width=800,
    height=500,
                 annotations=[
       go.layout.Annotation(
            showarrow=False,
            text='Source: World Bank',
            xanchor='right',
            x=35,
            xshift=275,
            yanchor='top',
            y=0.05,
            font=dict(
                family="Courier New, monospace",
                size=10,
                color="black"
            )
        )])
fig.update_traces(texttemplate='%{x:.2s}', textposition='outside')
fig.update_xaxes(showgrid=False, zeroline=False, title='Respondents (in Percentage)')
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()

<p style="text-align:justify;font-size:16px"> From figure 1.2 it is evident that machine learning algorithms such as Linear Regression, Decision trees are more popular or preferred compared to Deep Learning Algorithms. This could be because of the following reasons
    <ol style="font-size:16px">
  <li>Any machine learning course would start with supervised learning/ unsupervised learning which takes it to regression and classification in which linear regresssion and decision tress are beginner friendly algorithms. Since majority of the respondents are beginners they would be more exposed to them.</li>
  <li>The above point brings it to the fact that machine learning algorithms are less complex than deep learning algorithms</li>
  <li>Machine Learning requires less computing power or specialized hardwares than deep learning</li>
</ol>
    
</p>

In [ ]:
group_df = df_2021.groupby(['Q15'])['Q17_Part_1','Q17_Part_2','Q17_Part_3','Q17_Part_4','Q17_Part_5','Q17_Part_6',
 'Q17_Part_7','Q17_Part_8','Q17_Part_9','Q17_Part_10','Q17_Part_11'].count()
percent_df = ((group_df.T/group_df.T.sum().values))*100

percent_df = percent_df[['Under 1 year','1-2 years','2-3 years','3-4 years','4-5 years','5-10 years','10-20 years','20 or more years']]
percent_df.index = ['Linear/ <br> Logistic Regression', 'Decision Trees or <br> Random Forests', 'Gradient Boosting Machines', 'Bayesian Approaches', 'Evolutionary Approaches', 'Dense Neural Networks', 'Convolutional Neural Networks', 'Generative Adversarial Networks', 'Recurrent Neural Networks', 'Transformer Networks', 'None']
percent_df.T.round(1).iplot(kind='barh',gridcolor='white',theme='white',
                              barmode = 'stack',
                              title=' Fig 1.3 Years of using ML Methods and their ML Algorithms Preference',
                              xTitle='Respondents (in Percentage)',color = colors)

<p>
    <ol style="font-size:16px">
  <li>All respondents prefer linear and logistic regression irrespective of their ML experience </li>
  <li>Machine Learning algorithms as in whole are highly prefered</li>
  <li>Among deep learning algorithms Convolutional Neural Networks are highly used</li>
        <li> Around 6% of respondents whose ML experience is less than 1 year doesnt use any of them </li>
</ol></p>


In [ ]:
researchers = df2021[df2021['Q24_Part_5'] == 'Experimentation and iteration to improve existing ML models']
deepLearning = [researchers['Q17_Part_6'].count(),researchers['Q17_Part_7'].count(),researchers['Q17_Part_8'].count(),researchers['Q17_Part_9'].count(),researchers['Q17_Part_10'].count()]
deepLearning_methods = ['Dense Neural Network','Convolutional Neural Network','Generative Adversarial Networks','Recurrent Neural Networks','Transformer Networks','None']
deepLearning_percent = list(np.divide(deepLearning,len(researchers)))
deepLearning_percent  = [(element * 100).round(0).astype(int) for element in deepLearning_percent]
deepLearning_dict = dict(zip(deepLearning_methods,deepLearning_percent))
sorted_dict={}

sortedList=sorted(deepLearning_dict.values())

for sortedKey in sortedList:
    for key, value in deepLearning_dict.items():
        if value==sortedKey:
            sorted_dict[key]=value
            
deepLearning= pd.DataFrame(sorted_dict.items(),columns=['deepLearning', '% of respondents'])    

xx = deepLearning['deepLearning']
xx = [_ + "<br>(" +str(sorted_dict[_])+ "%)" for _ in xx]
yy = [""]*len(sorted_dict)
zz = deepLearning['% of respondents']
cc = ["#e60049", "#0bb4ff", "#50e991", "#e6d800", "#9b19f5", "#ffa300", "#dc0ab4", "#b3d4ff", "#00bfa0","#0230c7","#00c707"]

trace1 = go.Scatter(x = xx, y = [""]*len(xx), mode='markers', name="", marker=dict(color=colors, opacity=0.6, size = zz*2.5))
layout = go.Layout(barmode='stack', height=300, margin=dict(l=100), title=' Fig 1.4 Deep Learning Methods Used by People Whose Job Involves Experimentation and <br> Iteration to Improve Existing ML models',
                   legend = dict(orientation="h", x=0.1, y=1.15), plot_bgcolor='#fff', paper_bgcolor='#fff', 
                   showlegend=False)

fig = go.Figure(data=[trace1], layout=layout)
fig.update_xaxes(showgrid=False, zeroline=False,title = 'Respondents (in Percentage)')
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()

<p style="text-align:justify;font-size:16px"> 
For Figure 1.4 respondents whose prime job involves Experimentation and Interation to improve existing ML models are considered. This set of people tries various algorithms, tunes the hyper parameters to see if they could improve the existing solutions.
    
</p>
 <ol style="font-size:16px">
  <li>Convolutional Neural Networks are highly prefered </li>

  </ol>

<br>
<p style="text-align:justify;font-size:16px"> 
Tough we keep telling deep learning is gaining popularity in the recent years, solves any complex problems, more accuracy etc it is very evident that basic machine learning algorithms are more prefered or they are familiar with.
</p>

****
<p style="text-align:justify;font-size:16px"> 
Computer Vision and NLP are the technologies that enable computer/machines to perceive things. Over the years both computer vision and NLP has seen tremendous growth. <br></p>
    <center><img src="https://i.imgur.com/JxdPigO.png[/img]" width = 500></center>   


 <p style="text-align:justify;font-size:16px">   <i>Natural language processing</i> is the branch of artificial intelligence that deals with training a computer to understand, process, and generate language. Search engines, machine translation services, and voice assistants are all powered by the technology. In the AI hype cycle, Gartner has placed NLP in the 'Trough of Disillusionment' reaching the 'Plateau of Productivity' in 5 to 10 years. </p>


In [ ]:
q19_columns = generateColumnNames(19,5)
q19_columns.append('Q19_OTHER')
algos_2021 = sort_dictionary_by_percent(df_2021,q19_columns,q19_count)
nlp_methods = pd.DataFrame(algos_2021.items(),columns=['Algorithms', '% of respondents'])    
trace = go.Bar(y = nlp_methods["Algorithms"],x = nlp_methods['% of respondents'] ,
orientation='h',marker=dict(color=color))
layout = go.Layout(barmode = "group",width=1000, height=500, showlegend=False)  
fig = go.Figure(data = [trace], layout = layout)
fig.update_traces(texttemplate='%{x:.2s}', textposition='outside')
fig.update_layout(title='Fig 1.5 Natural Language Processing Usage', width=800,height=500)
fig.update_xaxes(showgrid=False, zeroline=False,title = 'Respondents (in Percentage)')
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()  

  <p style="text-align:justify;font-size:16px">   
  <ol style="font-size:16px">
      <li> Word Embedding techniques such as GLoVe, word2Vec are highly used this is dues to the fact that before any computation the text has to be represented in some form.</li>
         <li> 45% of them are using transformer language models such as GPT-3, BERT which is great!
             <li> 26% of them aren't using any of the NLP methods
     </ol>
</p>

In [ ]:
group_df = df_2021.groupby(['Q15'])['Q19_Part_1', 'Q19_Part_2', 'Q19_Part_3', 'Q19_Part_4', 'Q19_Part_5','Q19_OTHER'].count()
percent_df = ((group_df.T/group_df.T.sum().values))*100
percent_df = percent_df[['Under 1 year','1-2 years','2-3 years','3-4 years','4-5 years','5-10 years','10-20 years','20 or more years']]
percent_df.index = ['Word embeddings/vectors', 'Encoder-decoder models', 'Contextualized embeddings', 'Transformer language models', 'None', 'Other']
percent_df.T.round(1).iplot(kind='barh',gridcolor='white',theme='white',
                              barmode = 'stack',
                              title='Fig 1.6 Years of using ML Methods and their NLP Methods Preference',
                              xTitle='Respondents (in Percentage)',color=colors)

  <p style="text-align:justify;font-size:16px">   
  <ol style="font-size:16px">
      <li> Red Red everywhere! all of them majorly use word embeddings 
      <li> Again transfomer language models are used by all category respondent
      <li> 26% of respondents with experience under 1 year aren't using NLP methods which takes it to the point beginners aren't using the newer technology.
      <li> Something dubious is among all category of respondents the none percentage is more
     </ol>
</p>

****

<p style="text-align:justify;font-size:16px">   <i>Computer vision</i> is a form of artificial intelligence that trains computers to interpret and understand the visual world. By combining digital images from camera feeds and videos with deep learning, machines can quickly learn to accurately identify and classify disparate objects. In the AI hype cycle, Gartner has placed computer vision in the 'Trough of Disillusionment' reaching the 'Plateau of Productivity' in 2 to 5 years. Which implies powerful computer vision solutions are arriving much sooner to production.  
</p>

In [ ]:
q18_columns = generateColumnNames(18,6)
q18_columns.append('Q18_OTHER')
algos_2021 = sort_dictionary_by_percent(df_2021,q18_columns,q18_count)
cv_methods = pd.DataFrame(algos_2021.items(),columns=['Algorithms', '% of respondents'])    
trace = go.Bar(y = cv_methods["Algorithms"],x = cv_methods['% of respondents'] ,
orientation='h',marker=dict(color=color, opacity=0.6))
layout = go.Layout(barmode = "group",width=1000, height=500, showlegend=False)  
fig = go.Figure(data = [trace], layout = layout)
fig.update_traces(texttemplate='%{x:.2s}', textposition='outside')
fig.update_layout(width=800,height=500,title = 'Fig 1.7 Computer Vision Methods Usage')
fig.update_xaxes(showgrid=False, zeroline=False,title = 'Respondents (in Percentage)')
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()  

  <p style="text-align:justify;font-size:16px">   
  <ol style="font-size:16px">
      <li> Image classification methods are highly used. This could be because image classification is a primary problem in computer vision which has it's use in many use cases. </li>
         <li> 20% of them are using GANs which is the current trend in CV
             <li> 19% of them aren't using any of the CV methods which is a good sign when compared to NLP
     </ol>
</p>

In [ ]:
group_df = df_2021.groupby(['Q15'])['Q18_Part_1','Q18_Part_2','Q18_Part_3','Q18_Part_4','Q18_Part_5',
 'Q18_Part_6','Q18_OTHER'].count()
percent_df = ((group_df.T/group_df.T.sum().values))*100
percent_df = percent_df[['Under 1 year','1-2 years','2-3 years','3-4 years','4-5 years','5-10 years','10-20 years','20 or more years']]
percent_df.index = ['General purpose image/video tools', 'Image segmentation methods', 'Object detection methods', 'Image classification', 'Generative Networks', 'None', 'Other']
fig = percent_df.T.round(1).iplot(kind='barh',gridcolor='white',theme='white',
                              barmode = 'stack',
                              title='Fig 1.8 Years of using ML Methods and their Computer Vision Methods Preference',
                              xTitle='Respondents (in Percentage)',color = colors)

 <p style="text-align:justify;font-size:16px">   
  <ol style="font-size:16px">
      <li> Yellove! all of them  majorly use image classfication methods 
      <li> Every other methods also has a good amount of respondents using them
      <li> 15% of beginners doesn't use any CV methods
      <li> When compared to NLP, the None numbers are better for CV
     </ol>
</p>

<div>
    <h2><center style="background-color:#00878b; color:white;">  2. SPECIALIZED HARDWARES </center><a id="2"></a></h2>
</div>
<p style="text-align:justify;font-size:16px">   With the advent of Neural networks over the recent years, it has triggered a huge demand for specialised hardware in the field of Machine learning. The need for reducing Training time and inference time for Neural networks came as key challenge for Hardware designers. This challenge and growth of usage over years led to high competition in the field of Hardware manufacturing to cater the fast growing requirement from the Industry. This focuses on specialized hardwares to see are they really used by the commoners. </p>

<p style="text-align:justify;font-size:16px">Google introduced Tensor Processing Units  in the year 2016. TPUs, unlike GPUs, is custom-designed to deal with operations such as matrix multiplications in neural network training. </p>


In [ ]:
k = df_2021['Q13'].value_counts().sort_values()
k = (k/len(df_2021))*(100)
k = k.round(0).astype(int)
tpus= pd.DataFrame(k.items(),columns=['TPU', '% of respondents'])    

xx = tpus['TPU']
xx = [_ + "<br>(" +str(k[_])+ "%)" for _ in xx]
yy = [""]*len(k)
zz = tpus['% of respondents']

trace1 = go.Scatter(x = xx, y = [""]*len(xx), mode='markers', name="", marker=dict(color=colors, opacity=0.6, size = zz*2.5))
layout = go.Layout(barmode='stack', height=300, margin=dict(l=100), title='Fig 2.1 TPU Usage',
                   legend = dict(orientation="h", x=0.1, y=1.15), plot_bgcolor='#fff', paper_bgcolor='#fff', 
                   showlegend=False)

fig = go.Figure(data=[trace1], layout=layout)
fig.update_xaxes(showgrid=False, zeroline=False,title = 'Respondents (in Percentage)')
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()

<p style="text-align:justify;font-size:16px"> It is hard to see 63% of them aren't using TPUs. Also 2% of them have used it more than 25 times which takes it to the fact that people aren't using TPUs even though it's access is freely provided by Google through third  its Cloud TPU service as part of the Google Cloud Platform and through its notebook-based services Kaggle and Colaboratory. Let's zoom into the respondents who chose None. </p>

<p style="text-align:justify;font-size:16px">    </p>
​

In [ ]:
df12_nvidia = df2021[df2021['Q13'] == 'Never']
df12_nvidia = df12_nvidia[df12_nvidia['Q15'].notna()]
k = df12_nvidia['Q15'].value_counts().sort_values()
k = (k/len(df12_nvidia))*(100)
k = k.round(0)
df = pd.DataFrame(k)
df['experience'] = df.index
fig = px.treemap(df,path=["experience"],values = df['Q15'],labels = df['experience'],
                  color='experience', hover_data=['Q15'],
                  color_continuous_scale='RdBu',title = 'Fig 2.2 ML Experience of Respondents who doesnt use TPU')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

<p style="text-align:justify;font-size:16px"> The above tree map repesents the respondents who haven't used TPU. It is evident that beginners aren't using such TPUs. Its is good to see only a few percent of the experienced respondents aren't using. This makes it clear that beginners require more knowledge on existence of such specialized hardware.  </p>

In [ ]:
deep_learners = df2021[['Q11','Q12_Part_1','Q12_Part_2','Q12_Part_3','Q12_Part_4','Q12_Part_5','Q12_OTHER','Q13','Q17_Part_6','Q17_Part_7','Q17_Part_8','Q17_Part_9','Q17_Part_10']]
deep_learners.dropna(how = 'all',subset=['Q17_Part_6','Q17_Part_7','Q17_Part_8','Q17_Part_9','Q17_Part_10'],inplace=True)
deep_learners.replace('A deep learning workstation (NVIDIA GTX, LambdaLabs, etc)','A deep learning workstation',inplace=True)
deep_learners.replace('A cloud computing platform (AWS, Azure, GCP, hosted notebooks, etc)','A cloud computing platform ',inplace=True)

k = deep_learners['Q13'].value_counts().sort_values()
k = (k/len(deep_learners))*(100)
k = k.round(0).astype(int)
df =  pd.DataFrame(k.items(),columns=['TPU', '% of respondents'])   
circles = circlify.circlify(
    list(df['% of respondents']), 
    show_enclosure=False, 
    target_enclosure=circlify.Circle(x=0, y=0, r=4)
)

fig, ax = plt.subplots(figsize=(6,6))

ax.axis('off')

lim = max(
    max(
        abs(circle.x) + circle.r,
        abs(circle.y) + circle.r,
    )
    for circle in circles
)
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)

labels = list(df['TPU'])

for circle, label in zip(circles, labels):
    x, y, r = circle
    if label in ['Never']:
        ax.add_patch(plt.Circle((x, y), r*0.95, alpha=0.6, facecolor=color, edgecolor=color))
        plt.annotate(label, (x,y ) ,va='center', ha='center',  fontsize=30)
    else:
        ax.add_patch(plt.Circle((x, y), r*0.95, alpha=0.5, facecolor='#E6E6E6', edgecolor='#E6E6E6'))
        plt.annotate(label, (x,y ) ,va='center', ha='center',  fontsize=10)

plt.title("Fig 2.3 Respondents who use Deep Learning Methods and their TPU Usage")
plt.show()

<p style="text-align:justify;font-size:16px"> 
For Figure 2.3 respondents use deep learning algorithms are considered as it is only for them TPUs are more useful. It is sad to see most of them have never used TPUs. 
</p>

****

In [ ]:
df_2021.replace('A deep learning workstation (NVIDIA GTX, LambdaLabs, etc)','A deep learning workstation',inplace=True)
df_2021.replace('A cloud computing platform (AWS, Azure, GCP, hosted notebooks, etc)','A cloud computing platform ',inplace=True)

k = df_2021['Q11'].value_counts().sort_values()
k = (k/len(df_2021))*(100)
k = k.round(0).astype(int)
tpus= pd.DataFrame(k.items(),columns=['TPU', '% of respondents'])    

xx = tpus['TPU']
xx = [_ + "<br>(" +str(k[_])+ "%)" for _ in xx]
yy = [""]*len(k)
zz = tpus['% of respondents']

trace1 = go.Scatter(x = xx, y = [""]*len(xx), mode='markers', name="", marker=dict(color=colors, opacity=0.4, size = zz*2))
layout = go.Layout(barmode='stack',   title='Fig 2.4 Computing Platform Usage',
                   legend = dict(orientation="h", x=0.1, y=1.15), plot_bgcolor='#fff', paper_bgcolor='#fff', 
                   showlegend=False)

fig = go.Figure(data=[trace1], layout=layout)
fig.update_xaxes(showgrid=False, zeroline=False,title = 'Respondents (in Percentage)')
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()

<p style="text-align:justify;font-size:16px"> 
Among computing platforms laptop followed by personal computer is the majority. This can't be considered, as everybody would have mostly chosen their personal devices. But there are 1% respondents who doesn't use any such computing platforms even their personal device. Let's check the further plot to understand who are these 1%.
</p>

In [ ]:
df12_nvidia = df2021[df2021['Q11'] == 'None']
df12_nvidia = df12_nvidia[df12_nvidia['Q15'].notna()]
k = df12_nvidia['Q15'].value_counts().sort_values()
k = (k/len(df12_nvidia))*(100)
k = k.round(0)
df = pd.DataFrame(k)
df['experience'] = df.index
fig = px.treemap(df,path=["experience"],values = df['Q15'],labels = df['experience'],
                  color='experience', hover_data=['Q15'],
                  color_continuous_scale='RdBu',title = 'Fig 2.5 ML Experience of Respondents who doesnt use Computing Platforms')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

<p style="text-align:justify;font-size:16px"> 
    <ol style="font-size:16px">
<li>Figure 2.5 is quite obvious as majority of respondents who doesnt use any ML methods doesn't require computing platforms 
 <li> Something which is odd is the beginners, 29% of respondents with less than 1 year experience and 6% of respondents with 1-2 years experience are also not using any such computing platforms                                                                                   </ol>
</p>

In [ ]:
k = deep_learners['Q11'].value_counts().sort_values()
k = (k/len(deep_learners))*(100)
k = k.round(0).astype(int)
k = k.iloc[1:]
df =  pd.DataFrame(k.items(),columns=['TPU', '% of respondents'])   
circles = circlify.circlify(
    list(df['% of respondents']), 
    show_enclosure=False, 
    target_enclosure=circlify.Circle(x=0, y=0, r=4)
)

fig, ax = plt.subplots(figsize=(6,6))

ax.axis('off')

lim = max(max(
        abs(circle.x) + circle.r,
        abs(circle.y) + circle.r,)
    for circle in circles
)
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)

labels = list(df['TPU'])

for circle, label in zip(circles, labels):
    x, y, r = circle
    if label in ['A laptop']:
        ax.add_patch(plt.Circle((x, y), r*0.95, alpha=0.6, facecolor=color, edgecolor=color))
        plt.annotate(label, (x,y ) ,va='center', ha='center',  fontsize=30)
    else:
        ax.add_patch(plt.Circle((x, y), r*0.95, alpha=0.5, facecolor='#E6E6E6', edgecolor='#E6E6E6'))
        plt.annotate(label, (x,y ) ,va='center', ha='center',  fontsize=10)

plt.title("Fig 2.6 Respondents who use Deep Learning Methods and their Computing Platforms Usage")
plt.show()

<p style="text-align:justify;font-size:16px"> 
Again, For Figure 2.6 respondents use deep learning algorithms are considered as specialized hardwares are closely associated with them. Ignoring the obvious one laptops and desktops a good amount of them are using deep learning workstations and cloud computing platforms.
</p>

****
<p style="text-align:justify;font-size:16px"> 
This section focuses on specilized hardwares in general which comprises NVIDIA GPUs, Google Cloud TPUs, AWS Trainium Chips, AWS Inferentia Chips </p>

In [ ]:
q12_count = {
     'NVIDIA GPUs' : (df2021['Q12_Part_1'].count()),
    'Google Cloud TPUs': (df2021['Q12_Part_2'].count()),
    'AWS Trainium Chips' : (df2021['Q12_Part_3'].count()),
    'AWS Inferentia Chips' : (df2021['Q12_Part_4'].count()),
    'None' : (df2021['Q12_Part_5'].count()),
    'Other' : (df2021['Q12_OTHER'].count())
}
q12_columns = generateColumnNames(12,5)
q12_columns.append('Q12_OTHER')
hardwares = sort_dictionary_by_percent(df2021,q12_columns,q12_count)
hardware = pd.DataFrame(hardwares.items(),columns=['Hardware', '% of respondents'])    
xx = hardware['Hardware']
xx = [_ + "<br>(" +str(hardwares[_])+ "%)" for _ in xx]
yy = [""]*len(hardwares)
zz = hardware['% of respondents']

trace1 = go.Scatter(x = xx, y = [""]*len(xx), mode='markers', name="", marker=dict(color=colors, opacity=0.4, size = zz*2))
layout = go.Layout(barmode='stack',   title='Fig 2.7 Specialized Hardware Usage',
                   legend = dict(orientation="h", x=0.1, y=1.15), plot_bgcolor='#fff', paper_bgcolor='#fff', 
                   showlegend=False)
fig = go.Figure(data=[trace1], layout=layout)
fig.update_xaxes(showgrid=False, zeroline=False,title = 'Respondents (in Percentage)')
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()

<p style="text-align:justify;font-size:16px"> 
Again percentage of none is more than 50% when compared to usage of specilized hardwares. Let's have a look at who are these none.</p>

In [ ]:
df12_none = df2021[df2021['Q12_Part_5'] == 'None']
df12_none = df12_none[df12_none['Q15'].notna()]
k = df12_none['Q15'].value_counts().sort_values()
k = (k/len(df12_none))*(100)
k = k.round(0)
df = pd.DataFrame(k)
df['experience'] = df.index
fig = px.treemap(df,path=["experience"],values = df['Q15'],labels = df['experience'],
                  color='experience', hover_data=['Q15'],
                  color_continuous_scale='RdBu',title = 'Fig 2.8 ML Experience of Respondents who doesnt use Specialized Hardwares')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

<p style="text-align:justify;font-size:16px"> 
It is evident that beginners aren't using specialized hardware as they hold the major space in the tree map. </p>

In [ ]:
xx = [deep_learners['Q12_Part_1'].count(),deep_learners['Q12_Part_2'].count(),deep_learners['Q12_Part_5'].count(),deep_learners['Q12_OTHER'].count(),deep_learners['Q12_Part_3'].count(),deep_learners['Q12_Part_4'].count()]
yy = ['AWS Inferentia Chips','AWS Traininum Chips','Other','None','Google Cloud TPU','NVIDIA GPU']
xx  = list(np.divide(xx,len(deep_learners)))
xx  = [(element * 100).round(0).astype(int) for element in xx]
circles = circlify.circlify(
   list(xx), 
    show_enclosure=False, 
    target_enclosure=circlify.Circle(x=0, y=0, r=4)
)

fig, ax = plt.subplots(figsize=(6,6))
ax.axis('off')
lim = max(
    max(
        abs(circle.x) + circle.r,
        abs(circle.y) + circle.r,
    )
    for circle in circles
)
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)

labels = yy

for circle, label in zip(circles, labels):
    x, y, r = circle
    if label in ['NVIDIA GPU']:
        ax.add_patch(plt.Circle((x, y), r*0.95, alpha=0.6, facecolor=color, edgecolor=color))
        plt.annotate(label, (x,y ) ,va='center', ha='center',  fontsize=25)
    else:
        ax.add_patch(plt.Circle((x, y), r*0.95, alpha=0.5, facecolor='#E6E6E6', edgecolor='#E6E6E6'))
        plt.annotate(label, (x,y ) ,va='center', ha='center',  fontsize=10)

plt.title("Fig 2.9 Respondents who use Deep Learning Methods and their Specialized Hardware Usage")
plt.show()

<p style="text-align:justify;font-size:16px"> 
    <ol style="font-size:16px">
<li>Respondents who use deep learning algorithms are using NVIDIA GPU majorly followed by Google Cloud TPU which is a good sign.
    <li> AWS Traininum and AWS Inferentia Chips are the least prefered
        </ol>
</p>

<div>
    <h2><center style="background-color:#00878b; color:white;">  3. CLOUD COMPUTING </center><a id="3"></a></h2>
</div>

<p style="text-align:justify;font-size:16px"> The field of data science is varied, and today there are many different roles and responsibilities involved in the process. Data science work typically involves working with unstructured data, implementing machine learning (ML) concepts and techniques, generating insights. This process typically ends in a visual presentation of data-driven insights. Machine learning is a critical element of the process, but training ML models is often a time-consuming process that requires a lot of resources. In the past, gaining access to ML resources was difficult and expensive. Today, many cloud computing vendors offer resources for data science in the cloud. </p>


In [ ]:
q27a_columns = ['Q27_A_Part_1',
                        'Q27_A_Part_2',
                        'Q27_A_Part_3',
                        'Q27_A_Part_4',
                        'Q27_A_Part_5',
                        'Q27_A_Part_6',
                        'Q27_A_Part_7',
                        'Q27_A_Part_8',
                        'Q27_A_Part_9',
                        'Q27_A_Part_10',
                        'Q27_A_Part_11',
                        'Q27_A_OTHER']

responses_df_2021 = df_2021
q27a_count = {
    'Amazon Web Services (AWS)' : (responses_df_2021['Q27_A_Part_1'].count()),
    'Microsoft Azure': (responses_df_2021['Q27_A_Part_2'].count()),
    'Google Cloud Platform (GCP)' : (responses_df_2021['Q27_A_Part_3'].count()),
    'IBM Cloud / Red Hat' : (responses_df_2021['Q27_A_Part_4'].count()),
    'Oracle Cloud' : (responses_df_2021['Q27_A_Part_5'].count()),
    'SAP Cloud' : (responses_df_2021['Q27_A_Part_6'].count()),
    'Salesforce Cloud' : (responses_df_2021['Q27_A_Part_7'].count()),
    'VMware Cloud' : (responses_df_2021['Q27_A_Part_8'].count()),
    'Alibaba Cloud' : (responses_df_2021['Q27_A_Part_9'].count()),
    'Tencent Cloud' : (responses_df_2021['Q27_A_Part_10'].count()),
    'None' : (responses_df_2021['Q27_A_Part_11'].count()),
    'Other' : (responses_df_2021['Q27_A_OTHER'].count())
}

cloud_professional = sort_dictionary_by_percent(responses_df_2021,q27a_columns,q27a_count)
cloud_professional  = OrderedDict(reversed(list(cloud_professional.items())))
cloud_professional = pd.DataFrame(cloud_professional.items(), columns=['Tools', '% of respondents'])
trace = go.Funnel(
    y=cloud_professional["Tools"],
    x=cloud_professional['% of respondents'], marker=dict(color=color))

layout = go.Layout(barmode="group", width=1000, height=500, showlegend=False)
fig = go.Figure(data=[trace], layout=layout)
fig.update_layout(title='Fig 3.1 Cloud Computing PLatforms Usage', width=800,height=500)
fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()

<p style="text-align:justify;font-size:16px"> 
    <ol style="font-size:16px">
<li> AWS and GCP tops the list 
    <li> 28% of respondents doesn't use any cloud computing tools
        </ol>
</p>

In [ ]:
cloud_cost = df_2021[['Q15','Q26']]
cloud_cost.dropna(inplace=True)
group_df = pd.crosstab(cloud_cost['Q15'], cloud_cost['Q26'])
group_df = group_df[['$0 ($USD)','$1-$99','$100-$999','$1000-$9,999','$10,000-$99,999','$100,000 or more ($USD)']]
percent_df = ((group_df.T/group_df.T.sum().values))*100
percent_df.index = [' 0 USD','1-99 USD','100-999 USD','1000-9,999 USD','10,000-99,999 USD','>100,000 USD']
fig = percent_df.T.round(1).iplot(kind='barh',gridcolor='white',theme='white',
                              barmode = 'stack',
                              title='Fig 3.2 Years of using ML Methods and their Amount Spent on Cloud Computing ',
                              xTitle='Respondents (in Percentage)',color = colors)

<p style="text-align:justify;font-size:16px"> 
    <ol style="font-size:16px">
<li> Majority of beginners haven't spent on cloud computing platforms
    <li> On average, the mid experienced people have spent 1000 - 9999 USD
        </ol>
</p>

<div>
    <h2><center style="background-color:#00878b; color:white;">  4. AUTO ML </center><a id="4"></a></h2>
</div>

<p style="text-align:justify;font-size:16px">
Automated machine learning, also referred to as AutoML, is the process of automating the time-consuming, iterative tasks of machine learning model development. It allows data scientists, analysts, and developers to build ML models with high scale, efficiency, and productivity all while sustaining model quality. According to a report by Latentview, AutoML revenue is expected to reach 14,512 million dollars by 2030. </p>

In [ ]:
q36a_count = {
    'Automated data augmentation' : (df_2021['Q36_A_Part_1'].count()),
    'Automated feature engineering/selection': (df_2021['Q36_A_Part_2'].count()),
    'Automated model selection ' : (df_2021['Q36_A_Part_3'].count()),
    'Automated model architecture searches' : (df_2021['Q36_A_Part_4'].count()),
    'Automated hyperparameter tuning' : (df_2021['Q36_A_Part_5'].count()),
    'Automation of full ML pipelines' : (df_2021['Q36_A_Part_6'].count()),
    'None' : (df_2021['Q36_A_Part_7'].count()),
    'Other' : (df_2021['Q36_A_OTHER'].count())
}
q36a_columns = ['Q36_A_Part_1','Q36_A_Part_2','Q36_A_Part_3','Q36_A_Part_4','Q36_A_Part_5','Q36_A_Part_6','Q36_A_Part_7','Q36_A_OTHER']
automl_2021 = sort_dictionary_by_percent(df_2021, q36a_columns, q36a_count)
automl_2021  = OrderedDict(reversed(list(automl_2021.items())))
automl_methods = pd.DataFrame(automl_2021.items(), columns=['Tools', '% of respondents'])
trace = go.Funnel(
    y=automl_methods["Tools"],
    x=automl_methods['% of respondents'], marker=dict(color=color))

layout = go.Layout(barmode="group", width=1000, height=500, showlegend=False)
fig = go.Figure(data=[trace], layout=layout)
fig.update_layout(title='Fig 4.1 Auto ML Tools Usage', width=800,height=500)
fig.update_xaxes(showgrid=False, zeroline=False)
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()

<p style="text-align:justify;font-size:16px">
Tough autoML is evolving so rapidly figure shows autoML tools usage by 68% of the respondents is none which is contradicting! This could be because most of the respondents are beginners. Hence, a subset of the respondents can be chose for further analysis on autoML. </p>

In [ ]:
researchers = df2021[df2021['Q24_Part_5'] == 'Experimentation and iteration to improve existing ML models']
automl = [researchers['Q36_A_Part_1'].count(),researchers['Q36_A_Part_2'].count(),researchers['Q36_A_Part_3'].count(),researchers['Q36_A_Part_4'].count(),researchers['Q36_A_Part_5'].count(),researchers['Q36_A_Part_6'].count(),researchers['Q36_A_Part_7'].count(),researchers['Q36_A_OTHER'].count()]
automl_methods = ['Automated data augmentation','Automated feature engineering/selection','Automated model selection','Automated model architecture searches','Automated hyperparameter tuning','Automation of full ML pipelines','No / None','Other']
automl_percent = list(np.divide(automl,len(researchers)))
automl_percent  = [(element * 100).round(0).astype(int) for element in automl_percent]
automl_dict = dict(zip(automl_methods,automl_percent))

sorted_dict={}

sortedList=sorted(automl_dict.values())

for sortedKey in sortedList:
    for key, value in automl_dict.items():
        if value==sortedKey:
            sorted_dict[key]=value
            
automl= pd.DataFrame(sorted_dict.items(),columns=['Automl', '% of respondents'])    

xx = automl['Automl']
xx = [_ + "<br>(" +str(sorted_dict[_])+ "%)" for _ in xx]
yy = [""]*len(sorted_dict)
zz = automl['% of respondents']

trace1 = go.Scatter(x = xx, y = [""]*len(xx), mode='markers', name="", marker=dict(color=colors, opacity=0.6, size = zz*2.5))
layout = go.Layout(barmode='stack', height=300, margin=dict(l=100), title='Fig 4.2 Auto ML Methods Used by People Whose Job Involves Experimentation and <br> Iteration to Improve Existing ML models',
                   legend = dict(orientation="h", x=0.1, y=1.15), plot_bgcolor='#fff', paper_bgcolor='#fff', 
                   showlegend=False)

fig = go.Figure(data=[trace1], layout=layout)
fig.update_xaxes(showgrid=False, zeroline=False,title = 'Respondents (in Percentage)')
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()

<p style="text-align:justify;font-size:16px">
Respondents whose prime job involves experimentation and iteration to improve exisiting ML models are considered as the autoML methods mentioned would be highly useful for them fro tasks such as hyper tuning, running full experiment, feature engineering etc.  
<ol style="font-size:16px">
  <li>39% of them aren't using of the autoML methods.</li>
  <li>Remaining 61% of them are using atleast one of the autoML methods</li>
</ol>  </p>

<div>
    <h2><center style="background-color:#00878b; color:white;">  5. MACHINE LEARNING TOOLS </center><a id="5"></a></h2>
    </div>
    <p style="text-align:justify;font-size:16px">
Machine Learning tools and resources are available so that anyone can use technology to solve problems. It makes the process easier. Machine Learning has a vast array of tools, software, and platforms to train and deploy ML models, draw insights from data etc. Also, the technology keeps on advancing forward.  </p>


In [ ]:
analysis = df2021[df2021['Q24_Part_1'] == 'Analyze and understand data to influence product or business decisions']
k = analysis['Q41'].value_counts().sort_values()
k = (k/len(analysis))*(100)
k= k.round(0).astype(int)
tool= pd.DataFrame(k.items(),columns=['Tool', '% of respondents'])  
trace = go.Bar(y = tool["Tool"],x = tool['% of respondents'] ,
orientation='h',marker=dict(color=color, opacity=0.6))
layout = go.Layout(barmode = "group",width=1000, height=500, showlegend=False)  
fig = go.Figure(data = [trace], layout = layout)
fig.update_traces(texttemplate='%{x:.2s}', textposition='outside')
fig.update_layout(plot_bgcolor='white',width=800,height=500,title='Fig 5.1 Primary Tools Used by People Whose Job Involves Analyze and Understand Data')
fig.update_xaxes(showgrid=False, zeroline=False, title='Respondents (in Percentage)')
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()  

<p style="text-align:justify;font-size:16px">
Respondents whose job involves analyze and understand data to influence product or business decisions might require tools to gather, process, analyze, and visualize large volumes of past, current, and future data in order to generate actionable business insights, create interactive reports, and simplify the decision-making processes.
    <ol style="font-size:16px">
       <li> From figure most of them are using some sort of tool predominantly local developement environments and basic statistical softwares
           <li> Advanced tools  and newer business intelligence tools aren't prefered
    </ol>    
 </p>

<p style="text-align:justify;font-size:16px">
    
 </p>

In [ ]:
analysis = df2021[df2021['Q24_Part_1'] == 'Analyze and understand data to influence product or business decisions']

group_df = analysis.groupby(['Q24_Part_1'])['Q34_A_Part_1',
                        'Q34_A_Part_2',
                        'Q34_A_Part_3',
                        'Q34_A_Part_4',
                        'Q34_A_Part_5',
                        'Q34_A_Part_6',
                        'Q34_A_Part_7',
                        'Q34_A_Part_8',
                        'Q34_A_Part_9',
                        'Q34_A_Part_10',
                        'Q34_A_Part_11',
                        'Q34_A_Part_12',
                        'Q34_A_Part_13',
                        'Q34_A_Part_14',
                        'Q34_A_Part_15',
                        'Q34_A_Part_16',
                        'Q31_A_OTHER'].count()
percent_df = ((group_df.T/group_df.T.sum().values))*100
percent_df.index = ['Amazon Quicksight','Power BI','Google Data Studio','Looker','Tableau','Salesforce','Einstein Analytics','Qlik','Domo','TIBCO Spotfire','Alteryx','Sisense','SAP Analytics Cloud','Microsoft Azure Synapse','Thoughtspot','None','Other']
percent_df = percent_df.sort_values('Analyze and understand data to influence product or business decisions')
trace = go.Bar(y = list(percent_df.index),x = percent_df['Analyze and understand data to influence product or business decisions'] ,
orientation='h',marker=dict(color=color, opacity=0.6))
layout = go.Layout(barmode = "group",width=1000, height=500, showlegend=False)  
fig = go.Figure(data = [trace], layout = layout)
fig.update_traces(texttemplate='%{x:.2s}', textposition='outside')
fig.update_layout(width=800,height=500,title='Fig 5.2 Business Intelligence Tools Used by People Whose Job Involves Analyze and <br> Understand Data to Influence Product or Business Decisions')
fig.update_xaxes(showgrid=False, zeroline=False, title='Respondents (in Percentage)')
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()  

  <ol style="font-size:16px">
       <li> 24% aren't using any of the business intelligence tools but for people who does analysis to understand it would be highly useful
           <li> Tableau and Power BI are the popular among other tools
    </ol>

In [ ]:
researchers = df2021[df2021['Q24_Part_5'] == 'Experimentation and iteration to improve existing ML models']
ml_tools = [researchers['Q38_A_Part_1'].count(),researchers['Q38_A_Part_2'].count(),researchers['Q38_A_Part_3'].count(),researchers['Q38_A_Part_4'].count(),researchers['Q38_A_Part_5'].count(),researchers['Q38_A_Part_6'].count(),researchers['Q38_A_Part_7'].count(),researchers['Q38_A_Part_8'].count(),researchers['Q38_A_Part_9'].count(),researchers['Q38_A_Part_10'].count(),researchers['Q38_A_Part_11'].count(),researchers['Q38_A_OTHER'].count()]
ml_tools_methods = ['Neptune.ai','Weights and Biases','Comet.ml','Sacred + Omniboard','TensorBoard','Guild.ai','Polyaxon','Trains','Domino Model Monitor','MLflow','No / None','Other']
ml_tools_percent = list(np.divide(ml_tools,len(researchers)))
ml_tools_percent  = [(element * 100).round(0).astype(int) for element in ml_tools_percent]
ml_tools_dict = dict(zip(ml_tools_methods,ml_tools_percent))

sorted_dict={}

sortedList=sorted(ml_tools_dict.values())

for sortedKey in sortedList:
    for key, value in ml_tools_dict.items():
        if value==sortedKey:
            sorted_dict[key]=value
            
ml_tools= pd.DataFrame(sorted_dict.items(),columns=['Tools', '% of respondents'])    

trace = go.Bar(y = ml_tools["Tools"],x = ml_tools['% of respondents'] ,
orientation='h',marker=dict(color=color))
layout = go.Layout(barmode = "group",width=1000, height=500, showlegend=False)  
fig = go.Figure(data = [trace], layout = layout)
fig.update_traces(texttemplate='%{x:.2s}', textposition='outside')
fig.update_layout(width=800,height=500,title='Fig 5.3 ML Tools Used by People Whose Job Involves Experimentation and <br> Iteration to Improve Existing ML Models')
fig.update_xaxes(showgrid=False, zeroline=False, title='Respondents (in Percentage)')
fig.update_yaxes(showgrid=False, zeroline=False)
fig.show()  

<p style="text-align:justify;font-size:16px">
Figure 5.3 shows the ML tools that are used by respondents whose job involves experimentation and iteration to improve existing ML models. 
</p>
  <ol style="font-size:16px">
       <li> 37% of them aren't using any such special ML tools for their job
           <li> TensorBoard tops among the tools as it is popular could be used for deep learning experiments
    </ol>
    <br>
  <div>
    <h2><center style="background-color:#00878b; color:white;"> 6. THE QUEST TO LEARN </center><a id="6"></a></h2>
</div>   
<p style="text-align:justify;font-size:16px">
For most of the tools and technologies discussed above the majority is either none or any any one item which is popular among that group. Respondents aren't familiar with most of the new tools and technologies. Especially, the beginners! It is important for them to understand and make themselves familiar with various tools and technology. This section compares the respondents and their learning platform preference for improving. 
</p>

In [ ]:
researchers = df2021[df2021['Q24_Part_6'] == 'Do research that advances the state of the art of machine learning']
val = (researchers['Q42_Part_9'].count()/len(researchers))*100
val = val.round(0).astype(int)
fig_09_trace_02 =go.Figure( go.Indicator(
    domain = {'row' : 1, 'column' : 3},
    value = val,
    mode = 'gauge+number',
    title = {'text' : 'Fig 6.1 Journal Publications Preference by People whose Job Involves <br> Research that Advances the State of the Art of ML'},
    delta = {'reference' : 0},
    gauge = {'axis' : {'range' : [0, 100]},'bar': {'color': color},}))
fig_09_trace_02.show()

In [ ]:
beginners = ['Do not use machine learning methods','Under 1 year']
df_beginners = df_2021[df_2021.Q15.isin(beginners)]
q40_list_of_columns_2021 = ['Q40_Part_1',
                       'Q40_Part_2',
                       'Q40_Part_3',
                       'Q40_Part_4',
                       'Q40_Part_5',
                       'Q40_Part_6',
                       'Q40_Part_7',
                       'Q40_Part_8',
                       'Q40_Part_9',
                       'Q40_Part_10',
                       'Q40_Part_11',
                       'Q40_OTHER']

q42_list_of_columns_2021 = ['Q42_Part_1',
                       'Q42_Part_2',
                       'Q42_Part_3',
                       'Q42_Part_4',
                       'Q42_Part_5',
                       'Q42_Part_6',
                       'Q42_Part_7',
                       'Q42_Part_8',
                       'Q42_Part_9',
                       'Q42_Part_10',
                       'Q42_Part_11',
                       'Q39_OTHER']

q40_dictionary_of_counts_2021 = {
    'Coursera' : (responses_df_2021['Q40_Part_1'].count()),
    'EdX': (responses_df_2021['Q40_Part_2'].count()),
    'Kaggle Learn Courses' : (responses_df_2021['Q40_Part_3'].count()),
    'DataCamp' : (responses_df_2021['Q40_Part_4'].count()),
    'Fast.ai' : (responses_df_2021['Q40_Part_5'].count()),
    'Udacity' : (responses_df_2021['Q40_Part_6'].count()),
    'Udemy' : (responses_df_2021['Q40_Part_7'].count()),
    'LinkedIn Learning' : (responses_df_2021['Q40_Part_8'].count()),
    'Cloud-certification programs' : (responses_df_2021['Q40_Part_9'].count()),
    'University Courses' : (responses_df_2021['Q40_Part_10'].count()),
    'None' : (responses_df_2021['Q40_Part_11'].count()),
    'Other' : (responses_df_2021['Q40_OTHER'].count())
}
q42_dictionary_of_counts_2021 = {
    'Twitter (data science influencers)' : (responses_df_2021['Q42_Part_1'].count()),
    'Email newsletters (Data Elixir, OReilly Data & AI, etc)': (responses_df_2021['Q42_Part_2'].count()),
    'Reddit (r/machinelearning, etc)' : (responses_df_2021['Q42_Part_3'].count()),
    'Kaggle (notebooks, forums, etc)' : (responses_df_2021['Q42_Part_4'].count()),
    'Course Forums (forums.fast.ai, Coursera forums, etc)' : (responses_df_2021['Q42_Part_5'].count()),
    'YouTube (Kaggle YouTube, Cloud AI Adventures, etc)' : (responses_df_2021['Q42_Part_6'].count()),
    'Podcasts (Chai Time Data Science, OReilly Data Show, etc)' : (responses_df_2021['Q42_Part_7'].count()),
    'Blogs (Towards Data Science, Analytics Vidhya, etc)' : (responses_df_2021['Q42_Part_8'].count()),
    'Journal Publications (peer-reviewed journals, conference proceedings, etc)' : (responses_df_2021['Q42_Part_9'].count()),
    'Slack Communities (ods.ai, kagglenoobs, etc)' : (responses_df_2021['Q42_Part_10'].count()),
    'None' : (responses_df_2021['Q42_Part_11'].count()),
    'Other' : (responses_df_2021['Q42_OTHER'].count())
}

group_df = df_beginners.groupby(['Q15'])['Q40_Part_1',
                       'Q40_Part_2',
                       'Q40_Part_3',
                       'Q40_Part_4',
                       'Q40_Part_5',
                       'Q40_Part_6',
                       'Q40_Part_7',
                       'Q40_Part_8',
                       'Q40_Part_9',
                       'Q40_Part_10',
                       'Q40_Part_11',
                       'Q40_OTHER'].count()
percent_df = ((group_df.T/group_df.T.sum().values))*100
percent_df.index = ['Coursera', 'EdX', 'Kaggle Learn Courses', 'DataCamp', 'Fast.ai', 'Udacity', 'Udemy', 'LinkedIn Learning', 'Cloud-certification programs', 'University Courses', 'None', 'Other']
fig = percent_df.T.round(1).iplot(kind='barh',gridcolor='white',theme='white',
                              barmode = 'stack',
                              title='Fig 6.2 ML Beginners and their Learning Platforms Preference',
                              xTitle='Respondents (in Percentage)',color = colors)

In [ ]:
q42_list_of_columns_2021 = ['Q42_Part_1',
                       'Q42_Part_2',
                       'Q42_Part_3',
                       'Q42_Part_4',
                       'Q42_Part_5',
                       'Q42_Part_6',
                       'Q42_Part_7',
                       'Q42_Part_8',
                       'Q42_Part_9',
                       'Q42_Part_10',
                       'Q42_Part_11',
                       'Q39_OTHER']
group_df = df_beginners.groupby(['Q15'])['Q42_Part_1',
                       'Q42_Part_2',
                       'Q42_Part_3',
                       'Q42_Part_4',
                       'Q42_Part_5',
                       'Q42_Part_6',
                       'Q42_Part_7',
                       'Q42_Part_8',
                       'Q42_Part_9',
                       'Q42_Part_10',
                       'Q42_Part_11',
                       'Q42_OTHER'].count()
percent_df = ((group_df.T/group_df.T.sum().values))*100
percent_df.index = ['Twitter ', 'Email newsletters', 'Reddit', 'Kaggle', 'Course Forums', 'YouTube', 'Podcasts', 'Blogs', 'Journal Publications', 'Slack Communities', 'None', 'Other']
fig = percent_df.T.round(1).iplot(kind='barh',gridcolor='white',theme='white',
                              barmode = 'stack',
                              title='Fig 6.3 ML Beginners and their Media Sources Preference',
                              xTitle='Respondents (in Percentage)',color = colors)

<p style="text-align:justify;font-size:16px">
From figure 6.2 and 6.3 it is good to see the none percentage is very less and the respondents have some platform to keep themselves updated. Figure 6.1 is an example to check if the respondents whose job involves research that advances the state of art of ML are reading journal publications as only by reading various publications literature survey can be done. But only 33% of them are using journal publications.
</p>

 <div>
    <h2><center style="background-color:#00878b; color:white;">  KEY INSIGHTS </center> <a id="7"></a></h2>
</div>

  <ol style="font-size:16px">
   <li> For most of the categories discussed above the percentage of people who doesn't use them are high

   <li> When the None category is zoomed it is predominantly filled by beginners (less than 1 year or 1-2 years of ML experience)
   <li> This makes it clear beginners aren't using any new tools and technology
   <li> Respondents aren't using the sophisticated tools and technologies available for them that makes their job easier
   <li> They are aware of various learning platforms available and are using them

</ol>   

 <div>
    <h2><center style="background-color:#00878b; color:white;">  CONCLUSION </center><a id="8"></a></h2>
</div>
<p style="text-align:justify;font-size:16px">
As we look through the adoption of AI, advancements in core machine learning, computer vision, NLP are dominating usage and applications. But according to the survey results, the machine learning enthusiasts aren't much aware of all the recent developements or doesn't prefer them. Now we understand why this report is named as (Not) AI Hype Cycle 2021. A new technology always gets discovered, one doesn't has to expertise in all the tools and technologies but one can atleast experiment them through various resources available. This would help them build their profile if they are looking for a career in data science or even if they are already a part of data science community. 
</p>

****
### References
1. [Gartner Hype Cycle For AI 2021](https://www.gartner.com/en/articles/the-4-trends-that-prevail-on-the-gartner-hype-cycle-for-ai-2021)
2. [Gartner Hype Cycle For AI 2021 Glance by Kamal Mishra ](https://medium.com/@mishra.kamal/gartner-hype-cycle-for-ai-2021-3c11c1d80358)
3. [A survey of Deep Learning and its Applications](https://link.springer.com/article/10.1007/s11831-019-09344-w)
4. [AutoML is the future of Machine learning, LatentView](https://www.latentview.com/blog/is-automl-the-future-of-machine-learning/)
5. [Artificial Intelligence and Machine Learning for Beginners](https://digitate.com/blog/artificial-intelligence-and-machine-learning-for-beginners-part-1-of-3/)
6. [Data Science in the Cloud](https://www.kaggle.com/nitishabharathi/data-science-in-the-cloud-who-wins-the-cloud-war)
7. [2021 Kaggle Data Science & Machine Learning Survey, Paul Mooney](https://www.kaggle.com/paultimothymooney/2021-kaggle-data-science-machine-learning-survey)

****

<p style="text-align:justify;font-size:11px"> <i> All the insights discussed are with respect to only 2021 Kaggle Survey Data</i> </p>